# 5.5 读写文件
+ 有时我们希望保存训练的模型， 以备将来在各种环境中使用（比如在部署中进行预测）
+ 当运行一个耗时较长的训练过程时， 最佳的做法是定期保存中间结果， 以确保在服务器电源被不小心断掉时，我们不会损失几天的计算结果。

总结:
+ 1. 可以通过`load()`和`save()`来进行单个张量的保存
+ 2. 可以通过`save(net.state_dict(),'filename') ` `load_state_dict('filename')`进行整个模型参数的保存与读取

## 5.5.1 加载和保存张量
单个张量直接调用load和save函数分别读写，两个函数都需要我们提供一个名称？

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
# x作为输出保存到x-file文件当中
torch.save(x, 'x-file')

In [2]:
# 存储在文件中的数据读回内存
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

存储一个张量列表 然后读回内存

In [3]:
y = torch.arange(4)
torch.save([x,y],'./data/x-files')

In [4]:
# 读取张量列表
x2,y2 = torch.load('./data/x-files')
print(x2)
print(y2)


tensor([0, 1, 2, 3])
tensor([0, 1, 2, 3])


写入或读取字符串映射到张量的字典

In [5]:
mydict = {'x':x,'y':y}
torch.save(mydict,'./data/mydict')
mydict2 = torch.load('./data/mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0, 1, 2, 3])}

## 5.5.2 加载和保存模型的参数
深度学习框架提供了内置函数来保存和加载整个网络，但是保存仅仅保存参数而不是整个模型

In [6]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [10]:
print(net.state_dict())

OrderedDict([('hidden.weight', tensor([[-0.0672,  0.0381,  0.0085,  ...,  0.0638,  0.1145, -0.0556],
        [-0.1977, -0.0807,  0.0423,  ..., -0.2199,  0.2040, -0.0379],
        [ 0.1484,  0.0889, -0.0906,  ..., -0.2149,  0.0583, -0.1980],
        ...,
        [-0.1278,  0.0878,  0.2043,  ..., -0.0130,  0.2119, -0.0305],
        [-0.1637, -0.0405,  0.1935,  ..., -0.0667,  0.1848,  0.0622],
        [ 0.2220,  0.0347, -0.0784,  ...,  0.0210,  0.0171, -0.1268]])), ('hidden.bias', tensor([ 0.0749,  0.0924, -0.1360, -0.2092, -0.0994,  0.1444, -0.1062,  0.1258,
        -0.0841,  0.1795, -0.0471,  0.1253,  0.0656, -0.1549, -0.1015, -0.1067,
        -0.2080, -0.1775, -0.0849,  0.0214, -0.1719, -0.1077,  0.1326,  0.2154,
         0.2005, -0.0488,  0.1414,  0.2168, -0.1023,  0.0573, -0.0811,  0.0504,
        -0.1227, -0.1026, -0.1160, -0.2156, -0.0480,  0.2084, -0.0152, -0.1541,
         0.0188, -0.0053, -0.0516, -0.1625,  0.0876,  0.1302,  0.0270,  0.0454,
         0.0189, -0.1782, -0.1707, -0

In [7]:
# 模型的参数存储在一个叫做“mlp.params”的文件中
torch.save(net.state_dict(),'./data/mpl.params')

In [8]:
clone = MLP()
# 从文件当中加载参数
clone.load_state_dict(torch.load('./data/mpl.params'))
# 设置模型为评估模式 关闭一些计算 更快速更稳定
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [9]:
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])